<a href="https://colab.research.google.com/github/krystaldowling/MAST30034_Final_Project-/blob/CNN_Model/Standard_CNN_GridSearch_max100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This document contains code modified from https://realpython.com/python-keras-text-classification/. 

In [21]:
# Import statements
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [22]:
# Loading in preproccessed data
PATH = "/content/drive/My Drive/Data/"
data = pd.read_csv(PATH + "preproccessed_data.csv", lineterminator='\n')

data

,title,text,label
0,muslims busted stole millions gov ’ benefits,print pay back money plus interest entire fami...,1
1,attorney general loretta lynch plead fifth,attorney general loretta lynch plead fifth bar...,1
2,breaking weiner cooperating fbi hillary email ...,red state fox news sunday reported morning ant...,1
3,pin drop speech father daughter kidnapped kill...,email kayla mueller prisoner tortured isis cha...,1
4,fantastic trumps 7 point plan reform healthcar...,email healthcare reform make america great sin...,1
...,...,...,...
29449,process analytical instruments market – techna...,technavio published new report global process ...,0
29450,travel deals get 1200 air credit two apt cookt...,apt offering savings new cape york outback wil...,0
29451,taiwanese recyclers belief waste simply mispla...,taipei taiwan sept 8 2015 prnewswire recent ye...,0
29452,season curtain raiser ideal way honour john,blackburn sunday league john haydock memorial ...,0


In [23]:
# Performing test-train split
text_train, text_test, y_train, y_test = train_test_split(data['text'].values, data['label'].values, test_size=0.25, random_state=1000)

In [24]:
# Tokenizing data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(text_train)

X_train = tokenizer.texts_to_sequences(text_train)
X_test = tokenizer.texts_to_sequences(text_test)

# Vocab size- adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1  

In [25]:
# Padding sequences
maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [27]:
# Create Model
def create_model(num_filters, kernel_size, vocab_size, embedding_dim, maxlen):
    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Conv1D(num_filters, kernel_size, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(6, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [28]:
# Defining possible parameters 
param_grid = dict(num_filters=[32, 64, 128],
                  kernel_size=[3, 5, 7],
                  vocab_size= [vocab_size], 
                  embedding_dim=[100],
                  maxlen= [100])

In [29]:
# Run grid search on data
model = KerasClassifier(build_fn=create_model, epochs=6, batch_size=16)
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, cv=4, verbose=1, n_iter=4)
# Fitting Model
grid_result = grid.fit(X_train, y_train)

# Evaluate testing set
test_accuracy = grid.score(X_test, y_test)
print(test_accuracy)


Fitting 4 folds for each of 4 candidates, totalling 16 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/6
1036/1036 [==============================] - 188s 182ms/step - loss: 0.4543 - accuracy: 0.7848
Epoch 2/6
1036/1036 [==============================] - 188s 182ms/step - loss: 0.2716 - accuracy: 0.8980
Epoch 3/6
1036/1036 [==============================] - 188s 182ms/step - loss: 0.1158 - accuracy: 0.9652
Epoch 4/6
1036/1036 [==============================] - 188s 181ms/step - loss: 0.0422 - accuracy: 0.9896
Epoch 5/6
1036/1036 [==============================] - 188s 182ms/step - loss: 0.0222 - accuracy: 0.9948
Epoch 6/6
346/346 [==============================] - 1s 2ms/step - loss: 0.7822 - accuracy: 0.8289
Epoch 1/6
1036/1036 [==============================] - 188s 182ms/step - loss: 0.4419 - accuracy: 0.7940
Epoch 2/6
1036/1036 [==============================] - 188s 181ms/step - loss: 0.2602 - accuracy: 0.9058
Epoch 3/6
1036/1036 [==============================] - 188s 181ms/step - loss: 0.1279 - accuracy: 0.9587
Epoch 4/6
1036/1036 [==============================] - 188s 1

[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed: 298.2min finished


Epoch 1/6
1381/1381 [==============================] - 249s 180ms/step - loss: 0.4276 - accuracy: 0.8059
Epoch 2/6
1381/1381 [==============================] - 248s 180ms/step - loss: 0.2768 - accuracy: 0.8949
Epoch 3/6
1381/1381 [==============================] - 249s 180ms/step - loss: 0.1535 - accuracy: 0.9463
Epoch 4/6
1381/1381 [==============================] - 249s 180ms/step - loss: 0.0728 - accuracy: 0.9727
Epoch 5/6
1381/1381 [==============================] - 249s 181ms/step - loss: 0.0367 - accuracy: 0.9880
Epoch 6/6
461/461 [==============================] - 1s 2ms/step - loss: 0.7916 - accuracy: 0.8300
0.8299837112426758


In [30]:
# Save and evaluate results
s = ('Running data set\nBest Accuracy : {:.4f}\n{}\nTest Accuracy : {:.4f}\n\n')
output_string = s.format( grid_result.best_score_, grid_result.best_params_, test_accuracy)
print(output_string)

Running data set
Best Accuracy : 0.8288
{'vocab_size': 192727, 'num_filters': 128, 'maxlen': 100, 'kernel_size': 7, 'embedding_dim': 100}
Test Accuracy : 0.8300




In [31]:
# Using code modified from https://machinelearningmastery.com/how-to-calculate-precision-recall-f1-and-more-for-deep-learning-models/

pred = grid.predict(X_test)
pred = np.rint(pred)

# Precision Score
precision = precision_score(y_test, pred)
print('Test Precision: %f' % (precision*100))

# Recall Score
recall = recall_score(y_test, pred)
print('Test Recall: %f' % (recall*100))

# F1 Score
f1 = f1_score(y_test, pred)
print('Test F1 score: %f' % (f1*100))

Test Precision: 82.989691
Test Recall: 76.134931
Test F1 score: 79.414666
